In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [117]:
# creating empty lists top store the data.
coin_list = []
price_list = []
price_change_list = []
market_cap_list = []
market_cap_rank_list = []
volume_list = []
volume_rank_list = []
volume_change_list = []
circulating_supply_list = []
total_supply_list = []
diluted_market_cap_list = []
contracts_name_list = []
contracts_link_list = []
website_list = []
social_twitter_list = []
social_telegram_list = []

In [118]:
def fetch_coin_data(coin):
    url = f"https://coinmarketcap.com/currencies/{coin.lower()}/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        coin_data = {}

        # Extracting the price and growth elements data
        price_growth = soup.find('div', attrs={'class': 'sc-d1ede7e3-0 gNSoet flexStart alignBaseline'})
        
        if price_growth:
            price_tag = price_growth.find('span', attrs={'class': 'sc-d1ede7e3-0 fsQm base-text'})
            if price_tag:
                # Remove "$" and "," from the price
                price = price_tag.text.strip().replace("$", "").replace(",", "")
                coin_data['price'] = price

            # Extract the growth for both positive and negative changes
            growth_tag_positive = price_growth.find('p', attrs={'class': 'sc-71024e3e-0 sc-58c82cf9-1 bgxfSG iPawMI'})
            growth_tag_negative = price_growth.find('p', attrs={'class': 'sc-71024e3e-0 sc-58c82cf9-1 ihXFUo iPawMI'})

            if growth_tag_positive:
                # Handle positive change
                growth = growth_tag_positive.text.strip().replace("%\xa0(1d)", "")
                coin_data['price_change'] = f'+{growth}'
            elif growth_tag_negative:
                # Handle negative change
                growth = growth_tag_negative.text.strip().replace("%\xa0(1d)", "")
                coin_data['price_change'] = f'-{growth}'

        # Extracting market cap content data
        coin_content = soup.find('div', attrs={'class': "sc-d1ede7e3-0 jkmmuA content_folded"})
        if coin_content:
            market_cap_content = coin_content.find('div', attrs={'class':'sc-d1ede7e3-0 bwRagp'})
            if market_cap_content:
                market_cap_price_tag = market_cap_content.find('dd', attrs={'class': 'sc-d1ede7e3-0 hPHvUM base-text'})
                if market_cap_price_tag:
                    market_cap_price = market_cap_price_tag.text.strip()
                    # Extract only the number part from market cap and remove "$"
                    market_cap_price = market_cap_price.split('%')[1].replace("$", "").replace(",", "")
                    coin_data['market_cap'] = market_cap_price
                
                market_cap_rank_tag = market_cap_content.find('span', attrs={'class': 'text slider-value rank-value'})
                if market_cap_rank_tag:
                    market_cap_rank = market_cap_rank_tag.text.strip().replace("#", "")
                    coin_data['market_cap_rank'] = market_cap_rank

        # Extracting coin metrics
        coin_metrics_content = soup.find('dl', attrs={'class':'sc-d1ede7e3-0 bwRagp coin-metrics-table'})
        if coin_metrics_content:
            coin_metrics = coin_metrics_content.find_all('div', attrs={'class':'sc-d1ede7e3-0 bwRagp'})
            if len(coin_metrics) > 1:
                volume_24h_rank = coin_metrics[1].text.strip().split("%")[1]
                volume_24h_price, volume_rank_value = volume_24h_rank.split("#")
                
                # Remove "$" from volume and format properly
                volume = volume_24h_price.replace("$", "").replace(",", "")
                coin_data['volume'] = volume
                coin_data['volume_rank'] = volume_rank_value
                
                if len(coin_metrics) > 2:
                    Volume_Market_cap_24h = coin_metrics[2].text.strip().split()[-1].replace("%", "")
                    coin_data['volume_change'] = Volume_Market_cap_24h

                if len(coin_metrics) > 3:
                    Circulating_supply = " ".join(coin_metrics[3].text.strip().split()[2:])
                    Circulating_supply = Circulating_supply.split(" ")[0]
                    coin_data['circulating_supply'] = Circulating_supply.replace(",", "")

                if len(coin_metrics) > 4:
                    Total_supply = " ".join(coin_metrics[4].text.strip().split()[2:])
                    Total_supply = Total_supply.split(" ")[0]
                    coin_data['total_supply'] = Total_supply.replace(",", "")

                if len(coin_metrics) > 6:
                    fully_diluted_market_cap = coin_metrics[6].text.strip().split()[-1].replace("$", "")
                    coin_data['diluted_market_cap'] = fully_diluted_market_cap.replace(",", "")

        # Extracting the coin stats and info links
        info_links_section = soup.find('div', attrs={'class' : 'sc-d1ede7e3-0 cvkYMS coin-info-links'})
        if info_links_section:
            info_links = info_links_section.find_all('div', attrs={'class': 'sc-d1ede7e3-0 jTYLCR'})
            if len(info_links) > 0:
                contracts_details = info_links[0].text.strip().split()
                if len(contracts_details) > 1:
                    name = contracts_details[0].replace("Contracts", "").replace(':', "").lower()
                    address = contracts_details[1]
                    coin_data['contracts'] = [
                        {
                            "name" : name,
                            "address" : address,
                        }
                    ]

            if len(info_links) > 1:
                website_details = info_links[1].find('div', attrs = {'class':'sc-d1ede7e3-0 sc-7f0f401-0 gRSwoF gQoblf'})
                if website_details:
                    link_tag = website_details.find('a')
                    if link_tag:
                        link = link_tag['href']
                        coin_data['official_links'] = [
                            {
                                "name": "website",
                                "link" : link
                            }
                        ]

            if len(info_links) > 2:
                socials = info_links[2].find_all('a')
                if len(socials) > 1:
                    twitter = socials[0]['href']
                    telegram = socials[1]['href']
                    coin_data['socials'] = [
                        {
                            "name" : "twitter",
                            "url" : twitter
                        },
                        {
                            "name" : "telegram",
                            "url" : telegram
                        }
                    ]

        return coin_data
    else:
        return {'error': 'Failed to retrieve data'}

In [148]:
coins = ['duko', 'ethereum', 'tether', 'dogecoin', 'Toncoin']
coin = coins[4]
data = fetch_coin_data(coin = coin)
print(len(data))
data

13


{'price': '6.88',
 'price_change': '-2.26',
 'market_cap': '16736490470',
 'market_cap_rank': '9',
 'volume': '300675976',
 'volume_rank': '36',
 'volume_change': '1.80',
 'circulating_supply': '2431720427',
 'total_supply': '5107439973',
 'diluted_market_cap': '35152322399',
 'contracts': [{'name': 'ethereum', 'address': '0x582d...47def1'}],
 'official_links': [{'name': 'website', 'link': 'https://ton.org/'}],
 'socials': [{'name': 'twitter', 'url': 'https://twitter.com/ton_blockchain'},
  {'name': 'telegram', 'url': 'https://t.me/tonblockchain'}]}

In [149]:
# df_data = {
#     "Coin" : coin,
#     "Price" : data['price'],
#     "Price_Change" : data['price_change'],
#     "Market_Cap" : data['market_cap'],
#     "Market_Cap_Rank" : data['market_cap_rank'],
#     "Volume" : data['volume'],
#     "Volume_Rank" : data['volume_rank'],
#     "Volume_Change" : data['volume_change'],
#     "Circulating_Supply" : data['circulating_supply'],
#     "Total_Supply" : data['total_supply'],
#     "Diluted_Market_Cap" : data['diluted_market_cap'],
#     "Contract_Name" : data['contracts'][0].get('name'),
#     "Contract_Address" : data['contracts'][0].get('address'),
#     "Website" : data['official_links'][0].get('link'),
#     (data['socials'][0].get('name', 'N/A')).capitalize() : data['socials'][0].get('url', 'N/A'),
#     (data['socials'][1].get('name', 'N/A')).capitalize() : data['socials'][1].get('url', 'N/A'),
# }

In [150]:
# df = pd.DataFrame(df_data.items(), index=None)
# df

In [151]:
# df.to_csv("Coin_Data")

In [152]:
# adding data to the lists here.
coin_list.append(coin)
price_list.append(data['price'])
price_change_list.append(data['price_change'])
market_cap_list.append(data['market_cap'])
market_cap_rank_list.append(data['market_cap_rank'])
volume_list.append(data['volume'])
volume_rank_list.append(data['volume_rank'])
volume_change_list.append(data['volume_change'])
circulating_supply_list.append(data['circulating_supply'])
total_supply_list.append(data['total_supply'])
diluted_market_cap_list.append(data['diluted_market_cap'])
contracts_name_list.append(data['contracts'][0].get('name'))
contracts_link_list.append(data['contracts'][0].get('address'))
website_list.append(data['official_links'][0].get('link'))
social_twitter_list.append(data['socials'][0].get('url', 'N/A'))
social_telegram_list.append(data['socials'][1].get('url', 'N/A'))

In [153]:
df_data = {
    "Coin" : coin_list,
    "Price" : price_list,
    "Price_Change" : price_change_list,
    "Market_Cap" : market_cap_list,
    "Market_Cap_Rank" : market_cap_rank_list,
    "Volume" : volume_list,
    "Volume_Rank" : volume_rank_list,
    "Volume_Change" : volume_change_list,
    "Circulating_Supply" : circulating_supply_list,
    "Total_Supply" : total_supply_list,
    "Diluted_Market_Cap" : diluted_market_cap_list,
    "Contract_Name" : contracts_name_list,
    "Contract_Address" : contracts_link_list,
    "Website" : website_list,
    (data['socials'][0].get('name', 'N/A')).capitalize() : social_twitter_list,
    (data['socials'][1].get('name', 'N/A')).capitalize() : social_telegram_list,
}

In [154]:
df = pd.DataFrame(df_data)
df

,Coin,Price,Price_Change,Market_Cap,Market_Cap_Rank,Volume,Volume_Rank,Volume_Change,Circulating_Supply,Total_Supply,Diluted_Market_Cap,Contract_Name,Contract_Address,Website,Twitter,Telegram
0,duko,0.004107,-10.40,39692708,690,6265633,499,15.79,9663955990,9999609598,41071336,solana,HLptm5...2G7rf9,https://dukocoin.com/,https://twitter.com/dukocoin,https://t.me/+jlScZmFrQ8g2MDg8
1,ethereum,3543.05,-3.45,424651099160,2,15064210511,3,3.55,120155670,120155670,424651099160,bnb,Smart,https://github.com/ethereum/wiki/wiki/White-Paper,https://twitter.com/ethereum,https://reddit.com/r/ethereum
2,tether,0.9994,-0.03,112425963705,3,65781297697,1,58.51,112496265357,115086088089,115014167994,ethereum,0xdac1...831ec7,https://tether.to,https://twitter.com/tether_to,https://t.me/OfficialTether
3,dogecoin,0.1404,-2.68,20311246304,8,1000514821,12,4.93,144660626384,144660626384,20311246304,bnb,Smart,http://dogecoin.com/,https://twitter.com/dogecoin,https://reddit.com/r/dogecoin
4,Toncoin,6.88,-2.26,16736490470,9,300675976,36,1.80,2431720427,5107439973,35152322399,ethereum,0x582d...47def1,https://ton.org/,https://twitter.com/ton_blockchain,https://t.me/tonblockchain


In [155]:
# final csv
# df.to_csv("Coins_Final")